In [2]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.pandas as ps
from pyspark.sql.functions import lit,col

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Building dataset").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_04_01")
spark.stop()

/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [5]:

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'boruta'

In [4]:
def lasso_fs():
    result_df = pd.read_csv('Advanced_AF_DER_Dataset.csv', index_col=None)
    print(len(result_df))
    result_df['date'] = pd.to_datetime(result_df['date'])
    result_df['Year'] = result_df['date'].dt.year
    result_df['Month'] = result_df['date'].dt.month
    result_df['DayOfWeek'] = result_df['date'].dt.dayofweek
    X = result_df.drop(['Ticker', 'date', 'Implosion_Next_Year'], axis=1)
    cols = X.columns
    #print(X.head())
    Y = result_df['Implosion_Next_Year']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Initialize the Lasso regression model
    lasso_model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Fit the model on the training data
    lasso_model.fit(X_train, Y_train)
    
    # Get predictions on the test data
    predictions = lasso_model.predict(X_test)
    
    # Evaluate the model (optional)
    mse = mean_squared_error(Y_test, predictions)
    print(f'Mean Squared Error: {mse}')
    
    # Get feature coefficients
    coefficients = lasso_model.feature_importances_
    
    # Print the coefficients and corresponding feature names
    for feature, coef in zip(cols, coefficients):
        print(f'{feature}: {coef}')
    
    return predictions, coefficients
    
    
    

#     tscv = TimeSeriesSplit(n_splits=5)

#     for train_index, test_index in tscv.split(X):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

#         scaler = StandardScaler()
#         scaler.fit(X_train)
#         X_train_scaled = scaler.transform(X_train)
#         X_test_scaled = scaler.transform(X_test)

#         alpha = 0.01
#         lasso_model = Lasso(alpha=alpha)
#         lasso_model.fit(X_train_scaled, y_train)

#         feature_importances = lasso_model.coef_

#         feature_importance_df = pd.DataFrame({'Feature': cols, 'Importance': feature_importances})

#         feature_importance_df['Absolute Importance'] = feature_importance_df['Importance'].abs()
#         feature_importance_df = feature_importance_df.sort_values(by='Absolute Importance', ascending=False).drop('Absolute Importance', axis=1)
#         print(f'Feature Importances for Fold {tscv.get_n_splits()}:')
#         print(feature_importance_df)
        
        
def boruta_fs():
    result_df = pd.read_csv('Advanced_AF_DER_Dataset.csv', index_col=None)
    X = result_df.drop(['Ticker', 'date', 'Implosion_Next_Year'], axis=1)
    Y = result_df['Implosion_Next_Year']
    scaler = StandardScaler()
    scaler.fit(X)
    X=scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    model = xgb.XGBClassifier()
    feat_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)
    feat_selector.fit(X_train, y_train)
    
    print(feat_selector.support_)
    print(feat_selector.ranking_)
    
# preds, coefs = lasso_fs()
boruta_fs()
    
    


NameError: name 'BorutaPy' is not defined